In [1]:
import numpy as np
import scipy
import scipy.cluster.vq
import copy
import time
import pickle
#import readRoutine as rr
#import smecticsViz as smv
import os

In [2]:
class DirectorField(object):
    """Creates the layer-normal field class"""
    def __init__(self, N=64, Nz=8, L=4.0, K=0.04, K24=-0.06, A=0.0, F=0.0, bcs='planar', seed=None):

        self.N, self.Nz, self.L = N, Nz, L
        self.B, self.K, self.K24 = 1.0, K, K24
        self.components = ['x','y','z']
        self.k = self.kSpace()
        self.kSq = self.kSquare()

        if seed == None:
            self.seed = np.random.randint(100000)
        else:
            self.seed = seed
        np.random.seed( self.seed )

        self.bcs = bcs
        boundary_condition_dict = { 'none': self.nullAnchoring, 'planar': self.enforceStrictPlanarBCs,
                'homeotropic': self.enforceStrictHomeotropicBCs, 'mixed': self.enforceStrictMixedBCs }
        self.enforceBCs = boundary_condition_dict[ bcs ]

        self.omega = F
        self.amplitude = A#We've been picking A equals twice the gap size for horizontal shear.

        self.field = gaussianRandomField( self )

        self.enforceBCs()
       
        self.smoothenGaussian()
        
        self.field = unitField( self.field )
        curl_free_field = curlFreeProjection( self, self.field )
        cff_average = np.sqrt( squareField( curl_free_field ) ).mean()
        for c in self.components:
                self.field[c] = curl_free_field[c] / ( cff_average + 1e-16 )
        
        if bcs=='mixed':
            self.enforceBCs()

    def rspace(self):
        """ Returns r-vector dictionary. (N >= Nz)""" 
        r = {}
        for c in self.components:
            r[c] = np.zeros((self.N,self.N,self.Nz))
        for i in range(self.N):
            r['x'][i,:,:], r['y'][:,i,:] = i, i
            if i < self.Nz:
                r['z'][:,:,i] = i
        return r
        
    def kSpace(self):
        """ Returns 1BZ k-vector dictionary.
            Note that it sets k[..., N / 2, ...] = 0 so that derivatives are well-defined."""
        k = {}
        for c in self.components:
            k[c] = np.zeros((self.N,self.N,self.Nz))
        for i in range(1,self.N/2):
            k['x'][i,:,:] = 2.0 * np.pi * i / self.N
            k['x'][self.N-i,:,:] = - 2.0 * np.pi * i / self.N
            k['y'][:,i,:] = 2.0 * np.pi * i / self.N
            k['y'][:,self.N-i,:] = - 2.0 * np.pi * i / self.N
            if i < self.Nz/2:
                k['z'][:,:,i] = 2.0 * np.pi * i / self.Nz
                k['z'][:,:,self.Nz-i] = - 2.0 * np.pi * i / self.Nz    
        return k

    def kSquare(self):
        """ Returns k^2 array.
            Note that ksq[...,N/2,...] is not zero."""
        kWithHalf = copy.deepcopy(self.k)
        kWithHalf['x'][self.N/2,:,:] = 2.0 * np.pi * (1.0 / 2.0)
        kWithHalf['y'][:,self.N/2,:] = 2.0 * np.pi * (1.0 / 2.0)
        kWithHalf['z'][:,:,self.Nz/2] = 2.0 * np.pi * (1.0 / 2.0)        
        return kWithHalf['x']**2 + kWithHalf['y']**2 + kWithHalf['z']**2

    def smoothenGaussian(self):
        """ Updates (smoothen) field.field by applying Gaussian random filters.
            phi -> exp(-(L^2 k^2) / 4) phi in Fourier space."""
        normQ = np.sqrt( squareField( self.field ) ).mean()

        ffield = fourierTransform(self.field)
        for c in self.components:
            ffield[c] *= np.exp(- (self.L**2) * self.kSq / 4.0)
            self.field[c] = np.real(np.fft.ifftn(ffield[c]))

        """We project without changing the average norm"""
        normQ /= np.sqrt( squareField( self.field ) ).mean()
        for c in self.components:
            self.field[c] *= normQ         

    def nullAnchoring(self):
        """ This routine does not do anything."""

    def enforceStrictPlanarBCs(self):
        """ Enforces strict planar boundary conditions.
            N_z = 0 at 0 and Nz / 2, Nx (z) = Nx (-z), Ny (z) = Ny (-z), Nz (z) = - Nz (-z)"""
        self.field['z'][:,:,0] = 0.0
        self.field['z'][:,:,self.Nz / 2] = 0.0
        for i in range(1,self.Nz/2):
            self.field['x'][:,:,self.Nz/2+i] = self.field['x'][:,:,self.Nz/2-i]
            self.field['y'][:,:,self.Nz/2+i] = self.field['y'][:,:,self.Nz/2-i]
            self.field['z'][:,:,self.Nz/2+i] = -self.field['z'][:,:,self.Nz/2-i]

    def enforceStrictHomeotropicBCs(self):
        """ Enforces strict homeotropic boundary conditions.
            N_x, N_y = 0 at 0 and Nz / 2, N_x (z) = -N_x (-z), Ny (z) = - Ny (-z), Nz (z) = Nz (-z)"""
        self.field['x'][:,:,0] = 0.0
        self.field['x'][:,:,self.Nz / 2] = 0.0
        self.field['y'][:,:,0] = 0.0
        self.field['y'][:,:,self.Nz / 2] = 0.0
        for i in range(1,self.Nz/2):
            self.field['x'][:,:,self.Nz/2+i] = -self.field['x'][:,:,self.Nz/2-i]
            self.field['y'][:,:,self.Nz/2+i] = -self.field['y'][:,:,self.Nz/2-i]
            self.field['z'][:,:,self.Nz/2+i] = self.field['z'][:,:,self.Nz/2-i]

    def enforceStrictMixedBCs(self):
        """ Enforces strict homeotropic boundary conditions.
            Nx (2), Ny(2), Nz(2) = -Nx(1), -Ny(1), Nz(1) for (0, Nz/4) (mirrored)
            Nx (2), Ny(2), Nz(2) = Nx(1), Ny(1), -Nz(1) for (0, Nz/2) (mirrored) """
        self.field['x'][:,:,self.Nz / 4] = 0.0
        self.field['y'][:,:,self.Nz / 4] = 0.0
        self.field['z'][:,:,0] = 0.0
        self.field['z'][:,:,self.Nz / 2] = 0.0
        for i in range( 1, self.Nz / 4 ):
            self.field['x'][:,:,self.Nz/4+i] = -self.field['x'][:,:,self.Nz/4-i]
            self.field['y'][:,:,self.Nz/4+i] = -self.field['y'][:,:,self.Nz/4-i]
            self.field['z'][:,:,self.Nz/4+i] = self.field['z'][:,:,self.Nz/4-i]
        for i in range(1,self.Nz/2):
            self.field['x'][:,:,self.Nz/2+i] = self.field['x'][:,:,self.Nz/2-i]
            self.field['y'][:,:,self.Nz/2+i] = self.field['y'][:,:,self.Nz/2-i]
            self.field['z'][:,:,self.Nz/2+i] = -self.field['z'][:,:,self.Nz/2-i]

    def testStrictPlanarBCs(self):
        """ Tests strict planar boundary conditions.
            Enforces BCs if maximum error > 1e-10."""
        tp = {}
        for c in self.components:
            tp[c] = np.zeros((self.N,self.N,self.Nz/2))
        tp['z'][:,:,0]=self.field['z'][:,:,0] + self.field['z'][:,:,self.Nz/2]
        for i in range(1,self.Nz/2):
            tp['x'][:,:,i]=self.field['x'][:,:,i]-self.field['x'][:,:,self.Nz-i]
            tp['y'][:,:,i]=self.field['y'][:,:,i]-self.field['y'][:,:,self.Nz-i]
            tp['z'][:,:,i]=self.field['z'][:,:,i]+self.field['z'][:,:,self.Nz-i]
            err=max(abs(tp['x']).max(),abs(tp['y']).max(),abs(tp['z']).max())
        print("Strict planar BC errors: ", abs(tp['x']).max(), abs(tp['y']).max(), abs(tp['z']).max())
        if err > 1e-10:
            print("Resetting Boundary Conditions...")
            self.enforceStrictPlanarBCs()

    def scatteringAmplitudeII(self):
        """ Returns a scattering amplitude array.
            I = ( sin( 2 beta ) )^2 
            beta is the angle between n and the polarizer direction. """ 
        return ( self.field['x'] * self.field['y'] )**2 #/ ( squareField( self.field )**2 +1.e-16) 

    def scatteringAmplitude(self):
        """ Returns a scattering amplitude array.
            A = N_x * N_y"""
        return self.field['x'] * self.field['y'] #/ ( squareField( self.field ) + 1e-16 )

    def massDensity(self):
        """ Returns the mass density scalar function.
            Layers are obtained from contours of this function."""
        r = self.rspace()
        f_N = fourierTransform( self.field )
        kdotf = self.k['x'] * f_N['x'] + self.k['y'] * f_N['y'] + self.k['z'] * f_N['z']
        u = np.real( np.fft.ifftn( 1.0j * kdotf / ( self.kSq + 1e-16 ) ) )
        n0={'x': self.field['x'].mean(), 'y': self.field['y'].mean(), 'z': self.field['z'].mean()}
        return n0['x'] * r['x'] + n0['y'] * r['y'] + n0['z'] * r['z'] - u

    def gradientTensor(self):
        """ Returns the gradient tensor associated with the layer-normal field.
            D_{mu nu} = Nabla_mu N_nu """
        fourier_field = fourierTransform( self.field )
        gradient_tensor = {}
        for c1 in self.components:
            for c2 in self.components:
                gradient_tensor[c1, c2] = np.real( np.fft.ifftn( 1.j * self.k[c1] * fourier_field[c2] ) )
        return gradient_tensor

    def gradArg(self):
        """ Function used to test CUDA."""
        nsquare = squareField( self.field )
        dtensor = self.gradientTensor()
        return ( nsquare**2 ) * ( dtensor['x','x'] + dtensor['y', 'y'] + dtensor['z', 'z'] )

    def dislocationVector(self, gradient_tensor):
        """ Returns dislocation density vector.
            rho = curl N."""
        return {'x': gradient_tensor['y','z'] - gradient_tensor['z','y'], 
                'y': gradient_tensor['z','x'] - gradient_tensor['x','z'], 
                'z': gradient_tensor['x','y'] - gradient_tensor['y','x']}

    def testCurl(self):
        """ Tests if field is curl-free."""
        curlN = self.dislocationVector( self.gradientTensor() )
        print("Max components: ", abs(curlN['x']).max(), abs(curlN['y']).max(), abs(curlN['z']).max())

    def compressionEnergy(self):
        """ Returns compression energy density."""
        nsquare = squareField( self.field )
        return ( self.B / 4.0 ) * ( 1.0 - ( squareField( self.field )**2 ) )**2
        
    def splayEnergy(self):
        """ Returns splay energy density."""
        nsquare = squareField( self.field )
        dtensor = self.gradientTensor()
        trD = dtensor['x', 'x'] + dtensor['y', 'y'] + dtensor['z', 'z']
        return self.K * ( nsquare**2 ) * ( trD**2 )

    def saddleSplayEnergy(self):
        """ Returns saddle-splay energy density."""
        nsquare = squareField( self.field )
        dtensor = self.gradientTensor()
        dsquare = {}
        for c1 in self.components:
            for c2 in self.components:
                dsquare[c1,c2] = np.zeros((self.N,self.N,self.Nz))
                for c3 in self.components:
                    dsquare[c1,c2] += dtensor[c1, c3] * dtensor[c3, c2]
        trdsquare = dsquare['x', 'x'] + dsquare['y', 'y'] + dsquare['z', 'z']
        trd = dtensor['x', 'x'] + dtensor['y', 'y'] + dtensor['z', 'z']
        return ( self.K24 / 2.0 ) * ( nsquare**2 ) * ( trdsquare - ( trd**2 ) )

    def energyDensity(self):
        """ Returns elastic energy density."""
        energy = self.compressionEnergy() + self.splayEnergy() + self.saddleSplayEnergy() 
        #energy = ( self.compressionEnergy() + self.splayEnergy() + self.saddleSplayEnergy() 
        #        + self.dislocationCoreEnergy() ) 
        return energy

    def molecularField(self, gradient_tensor):
        """ Returns molecular field (apart from dislocation core contribution).
            h = delta F / delta N
            B, K, K24 ~ N^4, bulk ~ B (1 - N^4)
            Dislocation core contribution evaluated in molecularFieldDislocationCore."""
        k = self.k
        NFSq = squareField( self.field )
        D = gradient_tensor
        DSq = {}
        for c1 in self.components:
            for c2 in self.components:
                DSq[c1,c2] = np.zeros((self.N,self.N,self.Nz))
                for c3 in self.components:
                    DSq[c1,c2] += D[c1,c3] * D[c3,c2]
        trD = D['x', 'x'] + D['y', 'y'] + D['z', 'z']
        trDSq = DSq['x', 'x'] + DSq['y', 'y'] + DSq['z', 'z']
        
        grad_div = {} # = Nabla ( N^4 Tr D )
        ss_vec = {} # = [ D^2 - (Tr D) D ] . N
        temp = np.fft.fftn( ( NFSq**2 ) * trD )
        for c in self.components:
            grad_div[c] = np.real( np.fft.ifftn( 1.j * k[c] * temp ) )
            ss_vec[c] = np.zeros((self.N,self.N,self.Nz))
            for nu in self.components:
                ss_vec[c] += ( DSq[c, nu] - trD * D[c, nu] ) * self.field[nu] 

        molecular_field = {}
        for c in self.components:
            molecular_field[c] = np.zeros((self.N,self.N,self.Nz))
            molecular_field[c] += - 2.0 * self.B * NFSq * ( 1.0 - ( NFSq**2 ) ) * self.field[c]
            molecular_field[c] += - 2.0 * self.K * (grad_div[c] - 2.0 * NFSq * (trD**2) * self.field[c])
            molecular_field[c] += - 2.0 * self.K24 * NFSq * ( 2.0 * ss_vec[c] 
                    - ( trDSq - (trD**2) ) * self.field[c])

        return molecular_field

In [3]:
def gaussianRandomField( field, variance=1.0 ):
    """ Returns numpy gaussian random fields:
        P (N_mu) dN_mu = exp(- N_mu^2 / 2) dN_mu, for N_mu in [-infinity,infinity]"""
    return {'x': variance * np.random.randn(field.N, field.N, field.Nz),
            'y': variance * np.random.randn(field.N, field.N, field.Nz),
            'z': variance * np.random.randn(field.N, field.N, field.Nz)}

def laxCorrection( field ):
    """ Returns ( N_{i+1} + N_{i-1} + N_{j+1} + N_{j-1} + N_{k+1} + N_{k-1} ) / 6 - N_{i,j,k}
        for each component of field, where (i,j,k) are spatial indices.
        Used to cure spatial instabilities. """
    lax_fix = {}
    for c in field.components:
        lax_fix[c] = ( ( np.roll( field.field[c], 1, 0 )
                + np.roll( field.field[c], -1, 0 ) + np.roll( field.field[c], 1, 1 )
                + np.roll( field.field[c], -1, 1 ) + np.roll( field.field[c], 1, 2 )
                + np.roll( field.field[c], -1, 2 ) ) / 6.0
                - field.field[c] ) 
    return lax_fix

def numericalViscosity( field ):
    """ Returns a numerical viscosity term used to cure mesh driffiting."""
    numerical_viscosity = {}
    for c in field.components:
        numerical_viscosity[c] = ( np.roll( field.field[c], 1, 0 )
                + np.roll( field.field[c], -1, 0 ) + np.roll( field.field[c], 1, 1 )
                + np.roll( field.field[c], -1, 1 ) + np.roll( field.field[c], 1, 2 )
                + np.roll( field.field[c], -1, 2 )
                - 6.0 * field.field[c] ) 
    return numerical_viscosity

def squareField(vector_field):
    """ Returns the square of vector_field."""
    return vector_field['x']**2 + vector_field['y']**2 + vector_field['z']**2

def unitField(vector_field):
    """ Returns vector_field normalized to unit."""
    square_field = squareField( vector_field )
    return {'x': vector_field['x'] / ( np.sqrt( square_field ) + 1e-16 ),
            'y': vector_field['y'] / ( np.sqrt( square_field ) + 1e-16 ),
            'z': vector_field['z'] / ( np.sqrt( square_field ) + 1e-16 )}

def fourierTransform(vector_field):
    """ Returns the Fourier transform of vector_field."""
    return {'x': np.fft.fftn( vector_field['x'] ),
            'y': np.fft.fftn( vector_field['y'] ),
            'z': np.fft.fftn( vector_field['z'] )}

def curlFreeProjection(field, vector_field):
    """ Returns the curl-free component of vector_field.
        k-space Helmholtz decomposition:
        A_{cf} = F^{-1} [ k (k . F(A)) / k^2 ]"""
    ff = fourierTransform( vector_field )
    ff0 = {'x': ff['x'][0,0,0], 'y': ff['y'][0,0,0], 'z': ff['z'][0,0,0]}
    kdotf = field.k['x'] * ff['x'] + field.k['y'] * ff['y'] + field.k['z'] * ff['z']

    cf_vector_field = {}
    for c in field.components:
        ff[c] = field.k[c] * ( kdotf / (field.kSq + 1e-16) )
        ff[c][0,0,0] = ff0[c]
        cf_vector_field[c] = np.real(np.fft.ifftn(ff[c]))

    return cf_vector_field

def removeAverage( vector_field ):
    """ Removes the average value of a vector field. """
    return {'x': vector_field['x'] - vector_field['x'].mean(), 'y': vector_field['y']
            - vector_field['y'].mean(), 'z': vector_field['z'] - vector_field['z'].mean()}

def numericalViscosityStepper( field, time_step, time, molecular_field=None, gradient_tensor=None):
    """ Numerical viscosity stepper.
        Fixes mesh-drift instabilities.
        Beaware it fixes the time_step * viscous_constant. """
    viscous_constant = 0.1 #/ time_step
    f_field = fourierTransform( field.field )
    for c in field.components:
        field.field[c] = np.real( np.fft.ifftn( np.exp( - viscous_constant * ( field.kSq )**2
                * time_step ) * f_field[c] ) )

def gapStepper(field, time_step, time, molecular_field=None, gradient_tensor=None):
    """ Gap oscillation stepper.
        Updates field.field under gap oscillations.
        f(t) = t + 1 """
    f_prime_over_f = field.amplitude * field.omega * np.sin( field.omega * time) /( 1.0 + field.amplitude * ( 1.0 
            - np.cos( field.omega * time ) ) )
    #f_prime_over_f = field.amplitude * field.omega * np.cos( field.omega * time) /( 1.0
            #+ field.amplitude *  np.sin( field.omega * time  ) )
    field.field['x'] *= np.exp( f_prime_over_f * time_step / 2.0 )
    field.field['y'] *= np.exp( f_prime_over_f * time_step / 2.0 )
    field.field['z'] *= np.exp( -f_prime_over_f * time_step )

def shearStepper(field, time_step, time, molecular_field=None, gradient_tensor=None):
    """ Shear stepper.
        Updates field.field under shearing."""
    Lz = field.Nz / 2
    r = field.rspace()

    sign = np.zeros(field.field['x'].shape, dtype='float')
    sign[2*r['z'] > field.Nz] = 1
    sign[2*r['z'] < field.Nz] = -1

    v = field.amplitude * field.omega * np.abs( r['z'] / Lz - 1.0 ) * np.cos( field.omega * time )
    dvdz = field.amplitude * field.omega * sign * np.cos( field.omega * time ) / Lz

    nFx = {}
    for c in field.components:
        nFx[c] = np.exp( - 1.j * v * field.k['x'] * time_step ) * np.fft.fft( field.field[c], axis=0 )
        if c=='z':
            nFx[c] += ( np.exp( - dvdz * time_step ) - 1.0 ) * nFx['x']
        field.field[c] = np.real( np.fft.ifft( nFx[c], axis=0 ) )

def gradientDescentStepper(field, time_step, time, molecular_field, gradient_tensor=None):
    """ Curl-free gradient descent stepper.
        Updates field.field to minimize elastic energy."""
    #h_cf = curlFreeProjection( field, molecular_field )
    h_cf = removeAverage( molecular_field )
    #h_cf = molecular_field

    mesh_fix = False
    if mesh_fix:
        numericalViscosityStepper( field, time_step, time )

     #nUnit = unitField( field.field )
     #xi = np.random.randn()

    for c in field.components:
        field.field[c] += - h_cf[c] * time_step
        #rAct[c] += a * nUnit[c] * timestep + b * xi * np.sqrt( time_step)

    field.field = curlFreeProjection( field, field.field )

def operatorSplittingGap(field, time_step, time, molecular_field, gradient_tensor=None):
    """ Operator splitting.
        Updates field.field (gap + gradient descent) according to:
        U_B (t + delta/2, t + delta) U_A (t, t + delta) U_B (t, t + delta/2)"""
    gapStepper( field, time_step/2.0, time )
    gradientDescentStepper( field, time_step, time, molecular_field )
    gapStepper( field, time_step/2.0, time + time_step/2.0 )

def operatorSplittingShear(field, time_step, time, molecular_field, gradient_tensor=None):
    """ Operator splitting.
        Updates field.field (shear + gradient descent) according to:
        U_B (t + delta/2, t + delta) U_A (t, t + delta) U_B (t, t + delta/2)"""
    shearStepper( field, time_step/2.0, time )
    gradientDescentStepper( field, time_step, time, molecular_field )
    shearStepper( field, time_step/2.0, time + time_step/2.0 )
    #field.field = curlFreeProjection( field, field.field )

def adaptiveEulerStepper(field, initial_step_size, initial_time, approximate_final_time, atol, 
        stepper=gradientDescentStepper, verbose=False):
    """ Adaptive Euler stepper.
        Updates field.field, returns (finalTime, finalStepSize).
        To test CUDA use acceptRejectSimple."""
    h, t = initial_step_size, initial_time
    aerr0 = 1.0 * atol

    lax = False

    while t < approximate_final_time:
        OK = False

        D_1 = field.gradientTensor()
        molecular_field_1 = field.molecularField( D_1 )

        while not OK: 

            if lax:
                lax_fix = laxCorrection( field )

            A1, A2 = copy.deepcopy( field ), copy.deepcopy( field )

            stepper( A1, h, t, molecular_field_1, D_1 )

            stepper( A2, 0.5 * h, t, molecular_field_1, D_1 ) 

            D_2 = A2.gradientTensor()
            molecular_field_2 = A2.molecularField( D_2 )

            stepper( A2, 0.5 * h, t + 0.5 * h, molecular_field_2, D_2 ) 

            aerr = 0.;
            for c in field.components:
                aerr = max( [abs( A2.field[c] - A1.field[c] ).max(), aerr] )

            h, OK, aerr0 = acceptReject( atol, h, aerr, aerr0 )
            if verbose:
                print("t = ", t, ", h = ", h, ", err = ", aerr)
        t += h
        for c in field.components:
            field.field[c] = 2.0 * A2.field[c] - A1.field[c]

            if lax:
                field.field[c] = A1.field[c] + lax_fix[c]

    return t, h

def acceptRejectSimple(atol, h, aerr, aerr0=None):
    """ Accept or reject new field in adaptive Euler routine.
        Two-step Euler has error (K h^2) / 2 = |A_1 - A_2|.
        Accept if |A_1 - A_2| / h < tolerance."""
    OK = False
    if aerr < h * atol:
        OK = True
    h *= 0.9 * atol * h / aerr 
    return h, OK, aerr0

def acceptRejectSimplest(atol, h, aerr, aerr0=None):
    """ Accept or reject new field in adaptive Euler routine.
        Two-step Euler has error (K h^2) / 2 = |A_1 - A_2|.
        Accept if |A_1 - A_2| < tolerance."""
    OK = False
    if aerr <  atol:
        OK = True
    h *= 0.9 * np.sqrt( atol / aerr )
    return h, OK, aerr0

def acceptRejectStable(atol, h, aerr, aerr0):
    """ Accept or reject new field in adaptive Euler routine.
        Two-step Euler has error (K h^2) / 2 = |A_1 - A_2|.
        Improve stability.
        Accept if |A_1 - A_2| < tolerance."""
    OK = False
    beta = 0.3 # or 0.2 = 0.4 / 2.0
    alpha = 0.7 - 0.75 * beta
    if aerr <  atol:
        OK = True
    h *= 0.9 * ( ( atol / aerr )**alpha ) * ( ( aerr0 / atol )**beta )
    aerr0 = 1.0 * aerr
    return h, OK, aerr0

acceptReject = acceptRejectStable
#acceptReject = acceptRejectSimple # Use this one to test CUDA

def simpleEulerTest(field, stepsize, t):
    """ Routine designed to test current version of CUDA.
        Get rid of it after updating CUDA."""
    gradient_tensor = field.gradientTensor()
    molecular_field = field.molecularField( gradient_tensor )
    eulerCudaTest( field, stepsize, t , molecular_field, gradient_tensor )
    field.field = curlFreeProjection( field, field.field )

def osShearTest(field, time_step, time, molecular_field, gradient_tensor=None):
    """ Operator splitting test for CUDA.
        Get rid of it after updating CUDA."""
    shearStepper( field, 0.5 * time_step, time )
    eulerCudaTest( field, time_step, time, molecular_field )
    shearStepper( field, 0.5 * time_step, time + 0.5 * time_step )

def eulerCudaTest(field, stepsize, t, molecular_field, gradient_tensor=None):
    """ Routine designed to test current version of CUDA.
        Get rid of it after updating CUDA."""
    for c in field.components:
        field.field[c] += - molecular_field[c] * stepsize 

def adaptiveEulerIntegrator(field, initial_step_size, time_step, approximate_end_time, atol,
            stepper=gradientDescentStepper, verbose=False):
    """ Updates field.field from zero to end_time, using adaptive Euler method.
        Notice there is no save skip. We save after each time_step."""
    print("Running simulation...")
    routineTime = -time.clock()

    t, approximate_time = 0.0, 0.0
    step_size = initial_step_size

    saver, filename = rr.newFile( field )
    while t < approximate_end_time:
        t, step_size = adaptiveEulerStepper( field, step_size, t, approximate_time + time_step,
                atol, stepper, verbose )
        approximate_time += time_step
        field.enforceBCs() 
        rr.appendTimeField(saver, t, field)
        if verbose:
            print(t, field.energyDensity().sum())
    saver.close()

    routineTime += time.clock()
    print("It took this routine", routineTime / 60.0, "min.")

def simpleEulerIntegrator(field, time_step, save_skip, end_time, stepper=gradientDescentStepper,
            verbose=False):
    """ Updates field.field from zero to end_time, using simple Euler method."""
    print("Running simulation...")
    routineTime = -time.clock()
    t = 0.0
    saver, filename = rr.newFile( field )
    save_index = 1

    lax = False

    while t < end_time:
        if stepper != shearStepper:
            gradient_tensor = field.gradientTensor()
            molecular_field = field.molecularField( gradient_tensor )

        if lax:
            lax_fix = laxCorrection( field )

        stepper( field, time_step, t, molecular_field, gradient_tensor)

        if lax:
            for c in field.components:
                field.field[c] += lax_fix[c]

        t += time_step
        if save_index % save_skip ==0:
            field.enforceBCs()
            rr.appendTimeField(saver, t, field)
        save_index +=1
        if verbose:
            print(t)
    saver.close()
    
    routineTime += time.clock()
    print("It took this routine", routineTime / 60.0, "min.")

def setupCUDA(field, device=0, useNsq=0, doleslie=0):
    import smectics
    cuL = smectics.createDims(field.N, field.Nz)
    smectics.setupSystem(cuL, device)
    cuN = smectics.createDeviceArray(cuL, 3)

    cuSPHa = smectics.createDeviceArray(cuL, 1)
    flatSPHa = smectics.formatToDevice(field.noise_a)
    smectics.copyToDevice(cuL, 1, flatSPHa, cuSPHa)

    cuSPHb = smectics.createDeviceArray(cuL, 3)
    flatSPHb = smectics.formatToDevice(field.noise_b)
    smectics.copyToDevice(cuL, 3, flatSPHb, cuSPHb)

    ePerp, eParal, mxBC = 0, 0, 0
    newparam = field.defect_strength
    cuS = smectics.createSystem(field.B, field.K, field.K24, eParal, ePerp, mxBC, field.amplitude,
            field.omega, field.alpha, field.beta, field.gamma, 0.1, newparam, useNsq, doleslie, 0,
            cuSPHa, cuSPHb)
    return cuL, cuN, cuS

"""This is just to fix the Nsq typo in git repository"""
def evolutionCUDA(field, stepSize0, tStep, aTEnd, atol, cuL, cuN, cuS, stepper=0, imagesFolder="/b/smectics/"):
    import smectics
    routineTime = -time.clock()
    t, approxTime = 0.0, 0.0
    stepSize = stepSize0
    saver, filename = rr.newFile(field)
    eMin, eMax = 1.0, -1.0

    while(t < aTEnd):
        flatN = smectics.formatToDevice(field.field)
        smectics.copyToDevice(cuL, 3, flatN, cuN)
 
        t, stepSize = smectics.twoStepEuler(cuN, cuL, cuS, stepSize, t, approxTime + tStep, atol,
               stepper)

        smectics.copyFromDevice(cuL, 3, cuN, flatN)
        field.field = smectics.formatFromDevice(flatN, (field.N, field.N, field.Nz),
                            order=smectics.componentOrder3)

        field.enforceBCs()

        rr.appendTimeField(saver, t, field)

        #if saveTime % (5 * saveSkip) == 0:
            #sendMovies(field, filename, imagesFolder, t)

        approxTime += tStep
    saver.close()
    routineTime += time.clock()
    print("It took this routine", routineTime / 60.0, "min.")

def EulerStepCUDA(field, step, nsteps, cuL, cuN, cuS, verbose=False):
    import smectics
    flatN = smectics.formatToDevice(field.field)
    smectics.copyToDevice(cuL, 3, flatN, cuN)

    for i in xrange(int(nsteps)):
        print("Step ", i)
        smectics.simpleEuler(cuN, cuL, cuS, step)

        if i % 200 == 0:
            smectics.copyFromDevice(cuL, 3, cuN, flatN)
            field.field = smectics.formatFromDevice(flatN, (field.N, field.N, field.Nz),
                            order=smectics.componentOrder3)

            field.enforceBCs()

            flatN = smectics.formatToDevice(field.field)
            smectics.copyToDevice(cuL, 3, flatN, cuN)

        if i == 50 or i == 100 or i == 150:
            smectics.copyFromDevice(cuL, 3, cuN, flatN)
            field.field = smectics.formatFromDevice(flatN, (field.N, field.N, field.Nz),
                            order=smectics.componentOrder3)

            print("Energy check:", np.sum(field.energyDensity()))

            flatN = smectics.formatToDevice(field.field)
            smectics.copyToDevice(cuL, 3, flatN, cuN)

def totalSimulationTime(field, filename):
    """Print total time of a simulation file"""
    myFile = open(filename, 'rb')
    c1 = 0.0
    c2 = {}

    try:
        while True:
            c1 = pickle.load(myFile)
            c2 = pickle.load(myFile)
    except EOFError:
        print("End of file. Total time: %0.4e" % c1)


def loadFinalState(field, filename):
    """Return final state of a simulation file"""
    myFile = open(filename, 'rb')
    c1 = 0.0
    c2 = {}

    try:
        while True:
            c1 = pickle.load(myFile)
            c2 = pickle.load(myFile)
    except EOFError:
        print("End of file.")
    field.field = c2

def loadFieldAtT(field, instant, filename):
    """Recover field at instant t, for a given file"""
    myFile = open(filename, 'rb')
    t = 0.0
    while (t < instant):
        t = pickle.load(myFile)
        nField = pickle.load(myFile)
    field.field = nField
    myFile.close()